You need tables to write data into it. Make sure tor run Alchemy_tables befor running this.

# write to Database

In [29]:
cd C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5

C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5


In [30]:
from create_db import *

In [31]:
from sqlalchemy.exc import PendingRollbackError
from datetime import datetime
import os, re, json
from sqlalchemy.orm import sessionmaker, relationship

In [32]:
from dotenv import load_dotenv
from os import getenv
load_dotenv(r'C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5\.secrets')

True

In [33]:
pre_conn = getenv('pre_conn')
schema_name = getenv('DB_NAME')
connection_string = f'{pre_conn}{schema_name}'

In [34]:
engine = create_engine(connection_string, echo=False)

In [35]:
def convert_to_datetime(date_string: str) -> datetime:
    # this is for use in Video_class
    date_string = date_string.rstrip('Z')
    return datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S')

In [36]:
def iso_duration_to_seconds(iso_duration: str) -> int:
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', iso_duration)
    if not match:
        if iso_duration == "P0D":
            return 0
        else:
            raise ValueError(f"Invalid ISO 8601 duration format: {iso_duration}")
    hours = int(match.group(1) or 0)
    minutes = int(match.group(2) or 0)
    seconds = int(match.group(3) or 0)
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

In [37]:
Session = sessionmaker(bind=engine)
session = Session()

In [38]:
extracted_dir = r"C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5\extracted_data"

filepath = [
    'TheRegent-20250125-070247.json.json'
    'Behindwoods TV-20240825-115545.json',
    'GUVI-20240907-154356.json',
    'Sahi Siva-20240913-032253.json',
    'SHIVA SAI ENTERTAINMENT CHANNEL-20240824-190244.json'
]

In [39]:
for filename in filepath:
    print(f"Processing {filename}...")
    full_path = os.path.join(extracted_dir, filename)
    with open(full_path, 'r') as file:
        data = json.load(file)

    channel_data = data[next(iter(data))]
    channel_chid = channel_data.get("Channel_Id")
    CV = channel_data.get("Channel_Views")
    if not isinstance(CV, int):
        CV = int(CV)
    channel_record = Channel_class(
        channel_id=channel_chid,
        channel_name=channel_data.get("Channel_Name"),
        channel_type="Place_Holding_NoType",  # Placeholder
        channel_views=CV,
        channel_description=channel_data.get("Channel_Description"),
        channel_status=str(channel_data.get("Channel_Status"))
    )

    session.add(channel_record)
    try:
        session.commit()
    except PendingRollbackError:
        session.rollback()  # Roll back the session
        print("Transaction rolled back due to an error.")
    except Exception as e:
        session.rollback()  # Roll back on any other exception
        print(f"An error occurred: {e}")

    # Collect playlists and videos for batch insertion
    playlist_records = []
    video_records = []
    comment_records = []

    for playlist_data in channel_data.get("playlist", []):
        playlist_plid = playlist_data.get("playlist_ID")
        playlist_record = Playlist_class(
            playlist_id=playlist_plid,
            channel_id=channel_chid,
            playlist_name=playlist_data.get("playlist_title")
        )
        playlist_records.append(playlist_record)

        for videos_dict in playlist_data.get("videos", []):
            video_data = videos_dict[next(iter(videos_dict))]
            video_vid = video_data.get('Video_Id')
            video_record = Video_class(
                video_id=video_vid,
                playlist_ID=playlist_plid,
                video_name=video_data.get("Video_Name"),
                video_description=video_data.get("Caption_Status"),
                published_date=convert_to_datetime(video_data.get("PublishedAt")),
                view_count=video_data.get("View_Count"),
                like_count=video_data.get("Like_Count"),
                dislike_count=video_data.get("Dislike_Count"),
                favorite_count=video_data.get("Favorite_Count"),
                comment_count=video_data.get("Comment_Count"),
                duration=iso_duration_to_seconds(video_data.get("Duration")),
                thumbnail=video_data.get("Thumbnail"),
                caption_status=video_data.get("Caption_Status"),
            )

            if not session.query(Video_class).filter_by(video_id=video_vid).first():
                video_records.append(video_record)

            for comment_dict in video_data.get("Comments", {}).values():
                comment_cid = comment_dict.get("Comment_Id")
                comment_record = Comment_class(
                    comment_id=comment_cid,
                    video_id=video_vid,
                    comment_text=comment_dict.get("Comment_Text"),
                    comment_author=comment_dict.get("Comment_Author"),
                    comment_published_date=convert_to_datetime(comment_dict.get("Comment_PublishedAt"))
                )
                comment_records.append(comment_record)

    # Commit all records at once
    session.add_all(playlist_records + video_records + comment_records)
    try:
        session.commit()
    except PendingRollbackError:
        session.rollback()  # Roll back the session
        print("Transaction rolled back due to an error.")
    except Exception as e:
        session.rollback()  # Roll back on any other exception
        print(f"An error occurred: {e}")


Processing Behindwoods TV-20240825-115545.json...


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021DD7697680>>
Traceback (most recent call last):
  File "C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Elamparithi\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1513, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


An error occurred: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'UC8md0UEGj7UbjcZtMjBVrgQ' for key 'channels.PRIMARY'")
[SQL: INSERT INTO channels (channel_id, channel_name, channel_type, channel_views, channel_description, channel_status) VALUES (%(channel_id)s, %(channel_name)s, %(channel_type)s, %(channel_views)s, %(channel_description)s, %(channel_status)s)]
[parameters: {'channel_id': 'UC8md0UEGj7UbjcZtMjBVrgQ', 'channel_name': 'Behindwoods TV', 'channel_type': 'Place_Holding_NoType', 'channel_views': 13866375135, 'channel_description': 'Behindwoods covers Tamil Nadu & Chennai - News, Politics, Current Affairs, City Life, Tamil Movies, Personality Interviews, Movie Reviews, Events, Li ... (53 characters truncated) ... hindwoods is the most popular online media organisation in Tamil Nadu, trusted for its highest standards of ethics and quality for the past 19 years.', 'channel_status': "{'privacyStatus': 'public', 'isLinked': True, 'longUploadsStatus': 'longUploadsUnspecifi

TypeError: 'playlist_ID' is an invalid keyword argument for Video_class

## MySQL schema creation, Table creation and writing into it was completed.